# SBS: Tasa de interés promedio de las empresas bancarias

A continuación, se realizará un trabajo de scrapping para obtener información de las tasas de interés de créditos del sistema bancario. Esta información se registra en frecuencia diaria para diversas empresas bancarias operando en el mercado nacional peruano. 

In [8]:
import re
import time 
import json
import requests
import numpy as np
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
# Definimos una función que nos permite obtener el rango de fechas
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [10]:
# Creamos un loop para agrupar todas las fechas posibles
start_date = date(2015, 11, 24) # Año, mes, día
end_date = date(2021, 6, 8)
basemn = pd.DataFrame()
baseme = pd.DataFrame()
base = pd.DataFrame()
for single_date in daterange(start_date, end_date):
# Colocamos el formato de fecha reconocido por SBS
    interes = [single_date.strftime("%d/%m/%Y")]
    # interes = ['05/10/2022','06/10/2022']
    for interes in interes:
        try:
# URL
         url = "https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B"
# Buscador de google (abre la página automáticamente)
         chrome=Service("G:\Mi unidad\PROGRAMACIÓN APOYO\SBS\Chromedriver\chromedriver.exe")
         driver = webdriver.Chrome(service=chrome)
         driver.get(url)
# Seleccionar la fecha
         boton_fecha = driver.find_element('xpath', '//*[@id="ctl00_cphContent_rdpDate_dateInput"]')
         boton_fecha.clear()
         boton_fecha.send_keys(interes)
# Hacer clic en consulta
         boton_consulta = driver.find_element('xpath', '//*[@id="ctl00_cphContent_btnConsultar"]').click()
         time.sleep(5)

##### MONEDA NACIONAL #####
#-------------------------#
# Extracción de información
         mysoupme  = BeautifulSoup(driver.page_source, 'html.parser')
# Identificamos la tabla
         table1 = mysoupme.find('table', id="ctl00_cphContent_rpgActualMn_ctl00_DataZone_DT")
# Obtenemos todos los encabezados
         headersme = []
         for i in table1.find_all('th'):
                encabezadosme = i.text
                headersme.append(encabezadosme)
# Creamos un dataframe
         mydatame = pd.DataFrame(columns = headersme)
# Create un loop para completar todos los datos de tasas de interés
         for j in table1.find_all('tr')[1:]:
                row_datame = j.find_all('td')
                rowme = [i.text for i in row_datame]
                lengthme = len(mydatame)
                mydatame.loc[lengthme] = rowme
# me quedo solo con los datos promedio
         mydatame[interes] = mydatame['Promedio']
         mydatame = mydatame[interes]
# Transformo mis datos de series a Dataframe
         mydatame = mydatame.to_frame()
# Selecciono los índice y los valores por los que los voy a reemplazar
         mydatame = mydatame.iloc[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]]
         mydatame.index = ['Corporativos','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Grandes Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Medianas Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Pequeñas Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días',"Microempresas","Tarjetas de Crédito","Descuentos","Préstamos Revolventes","Préstamos a cuota fija hasta 30 días","Préstamos a cuota fija de 31 a 90 días","Préstamos a cuota fija de 91 a 180 días","Préstamos a cuota fija de 181 a 360 días","Préstamos a cuota fija a más de 360 días","Consumo","Tarjetas de Crédito","Préstamos Revolventes","Préstamos no Revolventes para automóviles","Préstamos no Revolventes para libre disponibilidad hasta 360 días","Préstamos no Revolventes para libre disponibilidad a más de 360 días","Créditos pignoraticios","Hipotecarios","Préstamos hipotecarios para vivienda"]
# Transpogo toda la información
         mydatame = mydatame.transpose()
         mydatame = mydatame.rename(index = lambda x: interes)
# Junto los diferentes resultados obtenidos
         baseme = baseme.append(mydatame)
         time.sleep(5)

##### MONEDA EXTRANJERA #####
#---------------------------#
         boton_extranjero = driver.find_element('xpath', '//*[@id="ctl00_cphContent_lbtnMex"]').click()
         time.sleep(5)
# Extracción de información
         mysoupmn  = BeautifulSoup(driver.page_source, 'html.parser')
# Identificamos la tabla
         table2 = mysoupmn.find('table', id="ctl00_cphContent_rpgActualMn_ctl00_DataZone_DT")
# Obtenemos todos los encabezados
         headersmn = []
         for i in table2.find_all('th'):
                encabezadosmn = i.text
                headersmn.append(encabezadosmn)
# Creamos un dataframe
         mydatamn = pd.DataFrame(columns = headersmn)
# Create un loop para completar todos los datos de tasas de interés
         for j in table2.find_all('tr')[1:]:
                row_datamn = j.find_all('td')
                rowmn = [i.text for i in row_datamn]
                lengthmn = len(mydatamn)
                mydatamn.loc[lengthmn] = rowmn
# me quedo solo con los datos promedio
         mydatamn[interes] = mydatamn['Promedio']
         mydatamn = mydatamn[interes]
# Transformo mis datos de series a Dataframe
         mydatamn = mydatamn.to_frame()
# Selecciono los índice y los valores por los que los voy a reemplazar
         mydatamn = mydatamn.iloc[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]]
         mydatamn.index = ['Corporativos','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Grandes Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Medianas Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Pequeñas Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días',"Microempresas","Tarjetas de Crédito","Descuentos","Préstamos Revolventes","Préstamos a cuota fija hasta 30 días","Préstamos a cuota fija de 31 a 90 días","Préstamos a cuota fija de 91 a 180 días","Préstamos a cuota fija de 181 a 360 días","Préstamos a cuota fija a más de 360 días","Consumo","Tarjetas de Crédito","Préstamos Revolventes","Préstamos no Revolventes para automóviles","Préstamos no Revolventes para libre disponibilidad hasta 360 días","Préstamos no Revolventes para libre disponibilidad a más de 360 días","Créditos pignoraticios","Hipotecarios","Préstamos hipotecarios para vivienda"]
# Transpogo toda la información
         mydatamn = mydatamn.transpose()
         mydatamn = mydatamn.rename(index = lambda x: interes)
# Junto los diferentes resultados obtenidos
         basemn = basemn.append(mydatamn)

# Convertimos las columnas string en float
         tipo = ['Corporativos','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Grandes Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Medianas Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días','Pequeñas Empresas','Descuentos','Préstamos hasta 30 días','Préstamos de 31 a 90 días','Préstamos de 91 a 180 días','Préstamos de 181 a 360 días','Préstamos a más de 360 días',"Microempresas","Tarjetas de Crédito","Descuentos","Préstamos Revolventes","Préstamos a cuota fija hasta 30 días","Préstamos a cuota fija de 31 a 90 días","Préstamos a cuota fija de 91 a 180 días","Préstamos a cuota fija de 181 a 360 días","Préstamos a cuota fija a más de 360 días","Consumo","Tarjetas de Crédito","Préstamos Revolventes","Préstamos no Revolventes para automóviles","Préstamos no Revolventes para libre disponibilidad hasta 360 días","Préstamos no Revolventes para libre disponibilidad a más de 360 días","Créditos pignoraticios","Hipotecarios","Préstamos hipotecarios para vivienda"]
         for tipo in tipo:
                basemn[tipo] = basemn[tipo].astype(float)
                baseme[tipo] = baseme[tipo].astype(float)
        except:
                pass
# Exportamos a excel
basemn.to_excel('Tasa Activa Moneda Nacional.xlsx', sheet_name='Moneda Nacional', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, inf_rep='inf', freeze_panes=None, storage_options=None)
baseme.to_excel('Tasa Activa Moneda Extranjera.xlsx', sheet_name='Moneda extranjera', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, inf_rep='inf', freeze_panes=None, storage_options=None)

C:\Users\MAURICIO\AppData\Local\Temp\ipykernel_9952\4093280931.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseme = baseme.append(mydatame)
C:\Users\MAURICIO\AppData\Local\Temp\ipykernel_9952\4093280931.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  basemn = basemn.append(mydatamn)
C:\Users\MAURICIO\AppData\Local\Temp\ipykernel_9952\4093280931.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseme = baseme.append(mydatame)
C:\Users\MAURICIO\AppData\Local\Temp\ipykernel_9952\4093280931.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  basemn = basemn.append(mydatamn)
C:\Users\MAURICIO\AppData\Local\Temp\ipykernel_9